In [ ]:
%matplotlib inline
import matplotlib
from ipywidgets import interactive, FloatSlider, HBox, interactive_output, Label as lbl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
from scipy.stats import t, norm
matplotlib.rcParams['figure.dpi'] = 144
import scipy.stats as stats
from bokeh.layouts import column
from bokeh.models import *
from bokeh.plotting import Figure, output_file, show
from bokeh.io import output_notebook, reset_output
from random import random, choice
from matplotlib import gridspec as gridspec
reset_output()
output_notebook()




In [ ]:
from scipy.stats import t as t

In [ ]:
import seaborn as sns
sns.set()


### Before starting, run all cells in the bottom of the notebook under the "plots" section

# Hypothesis Testing: Motivation

A media group offers a curated music selection service for restaurants and retail stores. Their sales team claims their curated playlists have a subconscious pyschological effect on customers which leads immediately to increased sales. It sounds hard to believe, they agree, but the results speak for themselves. 

Suppose a coffee shop is considering subscribing to a curated playlist service in their stores in hopes to increase sales revenue. They record the daily sales and 7 days of letting the employees choose music and then for 7 days of playing a free trial of the service.



In [ ]:
curated = np.array([ 971.04,  919.71,  853.42, 1020.38, 1000.92, 1062.83,  685.71])
non_curated = np.array([765.67, 897.53, 759.67, 999.32, 788.79, 838.69, 890.78])

coffee_sales = pd.DataFrame({"Curated":curated, 
                             "Non-curated":non_curated})
coffee_sales.index = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
coffee_sales['Differences'] = curated-non_curated
coffee_sales

From these measurements, can we conclude that the two treatments had different effects on sales? Computing their respective averages gives the following:

In [ ]:
curated_mean = curated.mean().round(2)
non_curated_mean = non_curated.mean().round(2)
difference_mean = (curated_mean-non_curated_mean).round(2)

print("\nAvg daily sales, curated playlist: ${}".format(curated_mean))
print("Avg daily sales, no curated playlist: ${}".format(non_curated_mean))
print("\nAvg difference in daily sales: ${}".format(difference_mean))

An observed difference in average sales of $\$81.93$ might seem like substantial evidence in favor of the curated playlist. But keep in mind that the recorded sales for each day is a random quantity, and so the average difference is a random quantity as well.

As a result, while the size of the difference does matter, we need to determine if it is the result of random chance or if it results from an underlying true difference between the two groups. 

To accomplish this, we will form a **hypothesis test** that uses known probability distributions to determine how likely we would have observed these results by chance, and report a statistical conclusion informed by this probability.

## Distributions and Density Functions

When we talk about random observations such as the ones we've described above, we're dealing mathematically with **random variables**, objects which take on values according to **probability distributions**. Instead of focusing on abstract definitions, let's consider some examples:

### Discrete Random Variables
Discrete variables take on values from a finite set of outcomes. For instance, if we flip a coin, we get one of two results. Similarly, if we roll a standard six-sided die, there are six possible results. In general, for either of these examples, we would consider each outcome to occur with equal probability. 

These are both examples of random variables with a *discrete uniform distribution*.

### Continuous Random Variables
Some random quantities can take on an infinite number of values. As an example, consider drawing a random number uniformly from the interval between zero and one. This number is a **continuous random variable**. Moreover it has a **uniform distribution** defined on the interval $[0,1]$. 

Continuous probability distributions are determined by a unique function called a **probability density function** (pdf). 

For instance, the normal distribution's density function has two parameters: $\mu$, the mean,  and $\sigma$, the standard deviation. Its definition is given below:

$$ f(x; \mu, \sigma) = \frac 1 {\sqrt{(2\pi\sigma^2)}} \exp\Big({\frac{-(x-\mu)^2}{2\sigma^2}}\Big)$$

At a glance, this might not seem very informative, so let's graph this function, letting $\mu=0$ and $\sigma^2 = 1$. 

Note: a normal distribution with  $\mu=0$ and $\sigma^2 = 1$ is called the **standard normal** distribution.


In [ ]:
normpdf_plot()

A common misconception is that a density function $f(x)$ gives us the probability of drawing the number $x$ from this distribution. In fact, if $X$ is a random variable taken from any continuous probability distribution, then for any number $x$, the probability of the random variable $X$ is equal to $x$, which we write as $P(X = x)$, is zero! 

We are able, however, to compute the probability that X lies in a particular region. For any random variable $X$ probability density function $f(x)$, the probability that $X$ is in the interval $[a,b]$ is given by taking the integral of $f(x)$ on the interval $[a,b]$, denoted by

$$ \int _a ^b f(x) dx$$

This is more clear from its visual representation: the area under the graph of $f(x)$ between $a$ and $b$. 

In [ ]:
t.cdf(2.45,6)-t.cdf(-2.45,6)

In [ ]:
normcdf_plot()

Instead of computing these integrals directly, we generally make use of a function known as the **cumulative density function**. This function tells us, for an input $x$,  the probability of drawing a random number less than or equal to $x$. We write the cumulative density function as

$$F(x) = \int_{-\infty}^x f(x) dx$$

We can use this function to get the integral of $f(x)$ on the interval $(a,b)$ as follows:

$$ \int _a ^b f(x) dx \: =  \int_{-\infty}^b f(x) dx \: \: - \:  \int_{-\infty}^a f(x) dx \: = \:  F(b)-F(a)$$

As a side note, this means it is also easy to obtain the probability that a random draw would be greater than x:

$$ P(X > x) = 1 - F(x)$$

We use the cdf to compute and visualize the probability that a random draw from a standard normal distribution would land in the interval $(-1.96, 1.96)$.

In [ ]:
norm.cdf(1.96)-norm.cdf(-1.96)

## A Simple Hypothesis Test

Now we have the tools to set up a simple hypothesis test.

Assume that $X = 1.75$ is a value drawn from a normal distribution with an unknown mean $\mu$ and a standard deviation of $\sigma = 1$. We want to prove that $\mu \not= 0$. 

Specifically, let's say we want to be $90\%$ sure that the true parameter $\mu$ is not equal to zero. In other words, we would like there to be only a $10\%$ probability that we make a mistake in concluding that $\mu\not=0$. Here we say that $90\%$ is the confidence level of the test, and $\alpha = 0.1 = 10\%$ is the error level.

### Z-test 
With these motivations in mind, we set up a **z-test** \*  with two competing hypotheses:

- **Null Hypothesis:**      $\mu=0$
- **Alternate Hypothesis:** $\mu\not=0$

A hypothesis test always proceeds as follows:
1. Assume the null hypothesis is true.
2. Compute rejection regions (as described below) for the test for the desired confidence level.
3. If the observed test statistic lies within the rejection regions, reject the null hypothesis.
    - For this simplified example, the test statistic is X.
    - For the coffee shop example, the test statistic will be the difference in average daily sales.

\* *We call this test a z-test because it is based on a standard normal distribution, and standard normal variables are often denoted by Z.*


### Rejection Regions

Now that we have a framework for testing, how do we determine the rejection regions for this test? Again, bear in mind that we want to prove that $\mu\not=0$. It stands to reason, then, that we would only want to reject the null hypothesis for values of X that are far from zero. 

Our alternate hypothesis, which claims $X \not= 0$, denotes that we are performing a 2-sided test. In general, the null hypothesis will be rejected based the absolute value of X, i.e. its distance from zero. Test rejection regions have boundary points given by **critical values**. For a 2-sided z-test, the critical value $x^*$ will be the value that satisfies

$$P(|x|\:  < \:|x^*|\: ) = 1 - \alpha$$

In other words, there will be a probability of $1-\alpha$ that a random draw from the null distribution will be in the interval $(-|x^*|, |x^*|)$, and a probability of $\alpha$ that it will be outside that interval. The rejection region of the test consists of all points outside that interval. Formally, the rejection region $R$ is defined by

$$ R = \{ x : |x| > |x^*| \} $$

Shown below is a plot summarizing the rejection regions and test results for a Normal(0,1) distribution based on a given value of $\alpha$. We can see how the results of our test change as we vary the value of $\alpha$.

In [ ]:
display(z_plot_interactive, ui_error)

# The power of a test

Our hypothesis test has two possible hypothesis and two possible conclusions. This means there are four possibilities to consider when running the test. These possible cases are displayed below in a *contingency table*:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:214px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-s268{text-align:left, font-size:200px}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-s268"></th>
      <th class="tg-s268"><b>Null Hypothesis is False</b></th>
      <th class="tg-0lax"><b>Null Hypothesis is True</b></th>
  </tr>
  <tr>
      <td class="tg-s268"><b>Reject Null</b></td>
    <td class="tg-s268">True Positive (TP)</td>
    <td class="tg-0lax">False Positive (FP)</td>
  </tr>
  <tr>
      <td class="tg-s268"><b>Fail to Reject Null</b></td>
    <td class="tg-s268">False Negative (FN) </td>
    <td class="tg-0lax">True Negative (TN) </td>
  </tr>
</table>
### Structure of a contingency table
- Columns represent the underlying truth
- Rows give the possible test results. 
- The values in the each cell are the probabilities of obtaining the result denoted by the row, given the truth condition stated by the column.

Using this terminology, we see that we define hypothesis tests by setting a rate on false positives. In other words, a test with $\alpha = 0.05$ means that there is a probability of $0.05$ of obtaining a false positive.

### Why do we start by limiting the probability of a false positive?
1. When/why are false positives less desirable than false negatives?
2. What happens if we start by trying to maximize the rate of true positives?


## Power

In statistical literature we call the true positive rate the **power** of the test, and denote it by $\beta$.

The value of $\beta$ is determined both by the value of $\alpha$ as well as the true value of the unknown parameter in the test. 

In our z-test, consider the following. If the true value of $\mu$ is $10$, we are more likely to reject the null hypothesis than if $\mu = 2$, because random draws when $\mu=10$ will tend to be farther away from zero. For a possible true value of $\mu$, we can compute the power of the test as the probability that a random draw from Normal($\mu, 1$) will fall within the rejection region $R$, which is given by

$$\int_R f(x;\mu, 1) dx$$






We visualize this below. The red bars at the bottom of the plot show the rejection regions determined by setting a value for $\alpha$, and the area of the shaded region gives the probability that X is within the rejection region based on the true value of $\mu$.

- What happens to $\beta$ as we vary $\mu$? What about when we vary $\alpha$?

In [ ]:
display(power_out,ui_power,  out)

### Precision and Recall

For those more familiar with data science, and especially for those familiar with information retrieval, the definition of test power may call to mind the concept of **recall** in terms of precision/recall. 

<img src="recall.jpg" width='300px'>

- Precision: Proportion of True Positives to (True Positives + False Positives)
- Precision: Proportion of True Positives to (True Positives + False Negatives)

In a hypothesis testing framework, the formulas for power and recall are the same, but there are slightly different assumptions or situations in which the precision/recall language is more appropriate.

With precision and recall, we're generally talking about a classifier, e.g. a yes or no answer to the question "is this example relevant." We are classifying data points from a sample into two groups based on that classifier. Crucially, while recall and power both give us the probability of obtaining a positive result, the consequences of said result are different when performing classification/document retrieval compared to testing.

With recall, a positive result means that we are including the data point in a class of data points. With hypothesis testing, a positive result means we are claiming that there is substantive evidence for the presence of an effect. 

# p-values

You might have noticed above that the Null Hypothesis is rejected for a certain range of $\alpha$ values. By moving the slider around, we see that the Null Hypothesis is rejected for approximately $\alpha \geq 0.09$, or equivalently, for up to 91% confidence.

If we had observed X = 1.68, on the other hand. We would have satisfied the rejection region for $\alpha = 0.1$ but not for $\alpha = 0.09$. With this in mind, it is reasonable to say that an observed value of $X=1.75$ gives *more evidence* for rejecting the null hypothesis compared to a value of $X=1.68$.

A **p-value** is a useful method for describing this idea numerically. Working from our graphical understanding of hypothesis testing, we can interpret the **p-value** of a test as the smallest value of $\alpha$ such that the null hypothesis is rejected. Mathematically, the p-value associated with the observed test statistic X is given by

$$ \text{p-value} = P(|x| > |X|), $$

where the probability is computed according to the null distribution.

Note: we can use p-values to determine the result of a hypothesis test. For a test with a given $\alpha$, if the computed p-value is less than $\alpha$, then we reject the null hypothesis.

In [ ]:
display(p_plot_interactive, ui_p, p_out)

## The Central Limit Theorem

Why are we talking about normal distributions in the first place? In terms of our motivating example, we don't know anything about the underlying distribution of daily sales differences. As it turns out, we can still make some assertions about the distribution of our average difference in sales.

The **Central Limit Theorem** (CLT) of statistics tells us that if $X$ is a random sample of length with true population mean $\mu$ and population standard deviation $\sigma$, and $\overline X$ is the mean of the sample $X$, then as n gets larger, the distribution of the quantity

$$ \frac{\overline x - \mu}{\sigma / \sqrt n} $$

will converge to a Normal distribution with mean 0 and variance $1$. In other words, for increasingly large sample sizes, if we know the population variance, it becomes increasingly reasonable to approximate the distribution of the sample mean as a normal random variable!

### Visualizing the CLT
From the above definition, the CLT can be hard to interpret intuitively. Here are some are some more effective (and visually appealing!) visualizations of the CLT to help get a better understanding.

http://mfviz.com/central-limit/

Central limit theorem for a binomial distribution
http://blog.vctr.me/posts/central-limit-theorem.html

# The t-distribution

The central limit theorem relies on the true population standard deviation. In a practical setting, we do not know this value, but we do have access to the sample standard deviation $s$, defined by

$$s = \frac{\sum (x_i - \overline x \:)^2}{n-1}$$

It turns out that we can still use the sample standard deviation to get an approximation of the distribution of $\overline x$ by substituting $s$ for $\sigma$ in the formula shown in the CLT, but this quantity is no longer converging to a normal distribution.

### Definition: 
Suppose that X is a random sample from a normal ($0, \sigma$) distribution. Then the distribution of

$$ \frac{\overline x - \mu}{s \: / \sqrt n} $$

is said to converge to a **t-distribution** with $n-1$ **degrees of freedom**. As it turns out, the t-distribution looks very similar to the normal distribution, but the t-distribution will have greater variance. Visually, there will be more density in the outer regions, or tails, of the t-distribution compared to the normal. 

In [ ]:
t_plot_1()

These "heavy tails" are not surprising when we consider the additional uncertainty in the t-distribution. Since we use an estimated value $s$ in constructing a t- random variable, it makes sense that we have less certainty in our results than if we used the true population standard deviation.

### Convergence of the t-distribution

As the sample size increases, we see values of $\overline x$ that will tend to be closer to the population mean, and we will also observe values of $s$ that will tend to be closer to the population standard deviation $\sigma$. 

As a result, as sample size increases, the t-distribution will become increasingly close to the standard normal distribution. As a result, the test rejection regions for the t-distribution will also converge to those for the normal distribution.

Shown below is a interactive plot which demonstrates the convergence of the t-distribution to the normal distribution, as well as the resulting confidence intervals. The red bars at the bottom of the plot show rejection regions for a 2-sided z-test, and the blue bars show the regions for the associated 2-sided t-test.

Note: as a result of this convergence, it is not uncommon to use a z-test instead of a t-test for large sample sizes. 

In [ ]:
display(ui, t_normal_out)

# Coffee Shop t-test

Finally, we're ready to return to our coffee shop daily sales example. Our test statistic is based the average difference in daily sales, which we will denote $\overline x$. We will be performing a t-test with 6 degrees of freedom and $\alpha = 0.05$.

Let's start by defining our hypotheses:
- Null Hypothesis: $\mu = 0$
- Alternate Hypothesis: $\mu \not= 0$

We can use the function t.ppf(quantile, df) to get the critical values for the test. Since we're performing a 2-sided test, we want to get the quantile for 2.5% and 97.5%, which will be opposites. 

In [ ]:
critical_value_positive = t.ppf(0.975, 6)
critical_value_positive

Finally, if the absolute value of our test statistic is greater than this critical value, we reject the null hypothesis!

In [ ]:
print("\nAvg daily sales, curated playlist: ${}".format(curated_mean))
print("Avg daily sales, no curated playlist: ${}".format(non_curated_mean))
print("\nAvg difference in daily sales: ${}".format(difference_mean))

In [ ]:
curated-non_curated
(curated-non_curated).mean()/((curated-non_curated).std()/(7**.5))

In [ ]:
s_differences = (curated-non_curated).std(ddof=1)
t_star = difference_mean/(s_differences/(7**.5))

print("Value of test statistic: {}".format(t_star))
print("Critical value of test: {}".format(critical_value_positive) + "\n")

if t_star > critical_value_positive:
    print("Reject the null hypothesis.")
else:
    print("Do not reject the null hypothesis.")

We can also compute the p-value associated with this test:

In [ ]:
p_val = 2*t.cdf(-abs(t_star), 6)
print("p-value: {}".format(p_val))

### The Easy way

It turns out that we don't have to reinvent the wheel any time we want to perform a t-test. There is actually a built-in t-test function in SciPy's stats module.

In [ ]:
scipy.stats.ttest_rel(curated, non_curated)

# Functions for this notebook's plots

In [ ]:
def normpdf_plot():
    # Define the range of x values
    x = np.linspace(-4, 4, 100)

    # norm.pdf gives the density of the normal distribution
    # here the loc argument is the mean, and the scale argument is the standard deviation
    y = norm.pdf(x)
    plt.figure(figsize = (3, 2))
    plt.plot(x,y)
    plt.suptitle("Density of the Standard Normal Distribution", fontsize = 8)
    plt.xlim(-4,4)
    plt.ylim(0, 0.42)
    plt.tick_params(axis='both', which='major', labelsize=6)
    plt.show()

In [ ]:
def normcdf_plot():
    #Define the range of x values
    x = np.linspace(-4, 4, 100)

    # norm.pdf gives the density of the normal distribution
    # here the loc argument is the mean, and the scale argument is the standard deviation
    y = norm.pdf(x, loc=0, scale=1)

    plt.figure(figsize = (3, 2))
    plt.fill_between(x, y, where=abs(x) < 1.96, alpha = 0.5)
    plt.ylim(0, 0.42)
    plt.annotate("Area of shaded \nregion: 0.95", (1, .3), fontsize = 8)
    plt.plot(x,y)
    plt.suptitle("P(-1.96 < X < 1.96)", fontsize = 8)
    plt.xlim(-4,4)
    plt.tick_params(axis='both', which='major', labelsize=6)
    plt.show()

In [ ]:
def z_test_plot(alpha):
    # Initialize plot and set title
    plt.figure(2, figsize=(16,6))
    plt.ylim(0, 0.41)
    plt.xlim(-4,4)
    plt.suptitle("Rejection Regions for Normal(mu, 1)" +
                 " with Null Hypothesis: mu=0",
                 fontsize=20)
    
    # Initialize x and y values
    x = np.linspace(-4, 4, 100)
    y = norm.pdf(x)
    
    # Draw normal curve
    plt.plot(x, y)
    
    # Compute 1-alpha/2 percentile
    r = norm.ppf(1-(alpha/2))
        
    # Results for rejecting null hypothesis
    if r < 1.75:
        # Draw dotted line for observed X value, color it green when rejecting null
        plt.plot([1.75]*2, [0, 0.41], linestyle = 'dotted', color = 'green', alpha = 0.7, linewidth = 3)
        plt.annotate("Reject Null Hypothesis", (1.84, 0.17), fontsize=13)
        plt.annotate("with {}% Confidence".format(int(100*(1- round(alpha, 2)))), (1.84, 0.15), fontsize=13)
        plt.annotate("(alpha = {})".format(round(alpha, 2)), (1.84, 0.13), fontsize=13)

    # Results for failing to reject null hypothesis
    else:
        # Draw dotted line for observed X value, color it green when failing to reject null
        plt.plot([1.75]*2, [0, 0.41], linestyle = 'dotted', color = 'red', alpha = 0.7)
        plt.annotate("Do Not Reject Null Hypothesis", (1.84, 0.17), fontsize=13)
        #plt.annotate("with {}% Confidence".format(int(100*(1- round(alpha, 2)))), (1.84, 0.15), fontsize=13)
        plt.annotate("(alpha = {})".format(round(alpha, 2)), (1.84, 0.15), fontsize=13)

    # Label the X = 1.75 line
    plt.annotate("X=1.75", (1.86, 0.2), fontsize=20)

    # Fill the rejection regions
    plt.fill_between(x, y, where=x < -r, interpolate=True, color='blue', alpha=0.5)
    plt.fill_between(x, y, where=x > r, interpolate=True, color='blue', alpha=0.5)
    
    # Disable y-axis tick marks and grid
    frame1 = plt.gca()
    frame1.axes.get_yaxis().set_visible(False)

    plt.show()
    


 # Create interactive plot with alpha value determined by a slider
alpha_plot1 = FloatSlider(min=0.01, max=0.2, step=0.01, continuous_update=False)
alpha_l = lbl(value=r"\(\alpha\)")
ui_error = HBox([alpha_l, alpha_plot1])
z_plot_interactive = interactive_output(z_test_plot, {'alpha': alpha_plot1})


In [ ]:
from ipywidgets import IntSlider, FloatSlider, interact, Output
from matplotlib import gridspec
def t_normal_plot(df, alpha):
    x = np.linspace(-6, 6, 200)
    y_norm = norm.pdf(x)
    y_t = t.pdf(x, df)
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[10, 1]) 
    
    fig = plt.figure(2,  figsize=(10,4))
    plt.xlim(-6, 6)
    plt.ylim(0, 0.41)
    plt.suptitle("{}% Rejection Regions for Normal(0,1) and \n".format(int(100*(1-alpha))) +  
                 "t-distributions with {} degrees of freedom".format(df),
                 fontsize = 12)
    
    ax = plt.subplot(gs[0])
    plt.ylim(-.01, 0.41)
    plt.xlim(-6, 6)

    tdist = plt.plot(x, y_t, color='blue', alpha = 0.5)
    Normal = plt.plot(x, y_norm, linestyle='dotted', color='red')
    
    r_norm = norm.ppf(1-(alpha/2))
    r_t = t.ppf(1-(alpha/2), df)
    y_r_t = t.pdf(r_t, df)

    plt.plot([-r_norm, -r_norm], [-0.05, norm.pdf(r_norm)], color='red', alpha = 0.8, linestyle  = 'dotted', linewidth = 1)
    plt.plot([r_norm, r_norm], [-0.05, norm.pdf(r_norm)], color='red', alpha = 0.8, linestyle  = 'dotted', linewidth = 1)

    plt.plot([-r_t, -r_t], [-0.2, y_r_t], color='blue', alpha = 0.8, linestyle  = 'dotted', linewidth = 1)
    plt.plot([r_t, r_t], [-0.2, y_r_t], color='blue', alpha = 0.8, linestyle  = 'dotted', linewidth = 1)

    plt.legend(["Normal(0,1)", "t-distribution"], fontsize = 6)
    plt.tick_params(axis='both', which='major', labelsize=4)
    frame1 = plt.gca()
    frame1.axes.get_yaxis().set_visible(False)
    plt.tick_params(axis='both', which='major', labelsize=8)

    ax2 = plt.subplot(gs[1])
    plt.ylim(-.1, 0)
    plt.xlim(-6, 6)

    plt.fill_between((-6, -r_norm), (-.05, -.05), -.025, color = "red", alpha = 0.4)
    plt.fill_between((r_norm, 6), (-.05, -.05), -.025, color = 'red', alpha = 0.4)

    plt.fill_between((-6, -r_t), (-.1, -.1), -.075, color = "blue", alpha = 0.4)
    plt.fill_between((r_t, 6), (-.1, -.1), -.075, color = 'blue', alpha = 0.4)

    plt.tick_params(axis='both', which='major', labelsize=4)


    frame1 = plt.gca()
    frame1.axes.get_xaxis().set_visible(False)
    frame1.axes.get_yaxis().set_visible(False)

    
    plt.show()

    
df = IntSlider(min=3, max=40, step=1, continuous_update=False)
alpha = FloatSlider(min=0.01, max=0.2, step=0.01, continuous_update=False)
df_l =  lbl(value="Degrees of Freedom")
alpha_l = lbl(value="alpha")
ui = HBox([df_l, df, alpha_l, alpha])

t_normal_out = interactive_output(t_normal_plot, {'df': df, 'alpha': alpha})

In [ ]:
def t_plot_1():
    x = np.linspace(-4, 4, 100)
    y_norm = norm.pdf(x)
    y_t = t.pdf(x, 5)
    plt.figure(figsize = (3, 2))
    plt.plot(x,y_norm, color='red', linestyle='dashed')
    plt.suptitle("Densities of Normal(0,1) and \n t-distribution with 5 degrees of freedom", fontsize = 7)
    plt.plot(x,y_t, color='blue')
    frame1 = plt.gca()
    frame1.axes.get_yaxis().set_visible(False)
    plt.tick_params(axis='both', which='major', labelsize=6)

    plt.legend(["Normal(0,1)", "t-distribution"], fontsize = 6)
    plt.show()

In [ ]:
p_out = Output(layout={'border': '2px solid gray', 'font-size': '20'})
with p_out:
    print("p-value: ")
def p_plot(X):
    # Initialize plot and set title
    plt.figure(2, figsize=(16,6))
    plt.ylim(0, 0.41)
    plt.xlim(-4,4)
    plt.suptitle("P-values", fontsize=20)
    
    # Initialize x and y values
    x = np.linspace(-4, 4, 200)
    y = norm.pdf(x)
    
    # Draw normal curve
    plt.plot(x, y)
    

    # Draw dotted line for observed X value, color it green when rejecting null
    plt.plot([X]*2, [0, 0.41], linestyle = 'dotted', color = 'green', alpha = 0.7, linewidth = 3)


    # Fill from X to infinity rejection regions
    plt.fill_between(x, y, where=x <= -abs(X), interpolate=True, color='blue', alpha=0.2)
    plt.fill_between(x, y, where=x >= abs(X), interpolate=True, color='blue', alpha=0.2)
    
    # Disable y-axis tick marks and grid
    frame1 = plt.gca()
    frame1.axes.get_yaxis().set_visible(False)
    p_out.clear_output(wait=True)
    with p_out:
        print("X = {}".format(X))
        print("p-value: {}".format(1-norm.cdf(abs(X)) + norm.cdf(-abs(X))))
    plt.show()
    


 # Create interactive plot with alpha value determined by a slider
X_p_plot = FloatSlider(value=1.75, min=-4, max=4, step=0.01, continuous_update=False)
X_l = lbl(value=r"Observed X value")
ui_p = HBox([X_p_plot, X_l])
p_plot_interactive = interactive_output(p_plot, {'X': X_p_plot})

In [ ]:
def confint_plot(): 
    X = np.random.normal(0,1,200).reshape(-1, 10)
    np.random.seed(23)

    fig = plt.figure()
    for c, num in zip([confint_norm(x, 0.1) for x in X], range(1,21)):
        ax = fig.add_subplot(21,1,num)
        plt.xlim(-1.5, 1.5)
        plt.axis('off')
        if c[0] > 0 or c[1] < 0:
            ax.plot(c, (0,0), 'r-')
            ax.plot(0, 0, 'ro')
        else:
            ax.plot(c, (0,0))
            ax.plot(0,0,'bo')
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
    ax2 = fig.add_subplot(21, 1, 21)
    plt.xlim(-1.5, 1.5)
    plt.ylim(0,0.01)
    plt.setp(ax2.get_yticklabels(), visible=False)
    fig.suptitle('Visualizing confidence levels on a 95% confidence interval', fontsize=10)
    sns.despine(offset=-5, trim=True, left = True)


In [ ]:
from matplotlib import gridspec
from ipywidgets import IntSlider, FloatSlider, interact, Output
out = Output(layout={'border': '2px solid gray', 'font-size': '20'})
with out:
    print("Power of Test: ")

def power_plot(mu, alpha):
    x = np.linspace(-6, 6, 200)
    y_null = norm.pdf(x)
    
    y_alternate = norm.pdf(x, mu, 1)

    
    fig = plt.figure(2,  figsize=(12,4))
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[20, 1]) 

    plt.xlim(-6, 6)
    plt.ylim(0, 0.41)
    
    ax1 = plt.subplot(gs[0])
    null = plt.plot(x, y_null, linestyle='dotted', color='red')
    alternate = plt.plot(x, y_alternate, color='blue')
    plt.legend(["Null Distribution", "True Distribution"], fontsize = 12)

    plt.suptitle("Power of a Hypothesis Test: \n" +
                 "Probability of a True Positive", fontsize = 12)
    
    r_norm = norm.ppf(1-(alpha/2))


    plt.plot([-r_norm, -r_norm], [-0.05, norm.pdf(r_norm)], color='red',
             alpha = 0.3, linestyle  = 'dotted', linewidth = 1)
    plt.plot([r_norm, r_norm], [-0.05, norm.pdf(r_norm)], color='red',
             alpha = 0.3, linestyle  = 'dotted', linewidth = 1)
    

    plt.fill_between(x, y_alternate, where=x<=-r_norm, color='blue', alpha=0.2)
    plt.fill_between(x, y_alternate, where=x>=r_norm, color='blue', alpha=0.2)
    
    
    plt.tick_params(axis='both', which='major', labelsize=4)

    frame1 = plt.gca()
    frame1.axes.get_yaxis().set_visible(False)
    plt.xlim(-6, 6)

    
    ax2 = plt.subplot(gs[1])
    ax2.axis("off")
    plt.fill_between((-6, -r_norm), (-.05, -.05), -.025, color = "red", alpha = 0.4)
    plt.fill_between((r_norm, 6), (-.05, -.05), -.025, color = 'red', alpha = 0.4)

    plt.xlim(-6, 6)

    out.clear_output(wait=True)
    
    power = (1-norm.cdf(r_norm, mu, 1)) +norm.cdf(-r_norm, mu, 1)
    with out:
        #null_lbl = lbl(value=r'Null Hypothesis: \(\mu = 0\)'.format(alpha))
        
        #mu_lbl = lbl(value=r'True value of \(\mu\): {}'.format(round(mu, 2)))
        #alpha_lbl = lbl(value=r'Error Level: \(\alpha\) = {}'.format(alpha))
        beta_lbl = lbl(value=r'Test Power: \(\beta\) = {}'.format(round(power, 2)))
        #display(HBox((null_lbl, mu_lbl, alpha_lbl, beta_lbl)))
        display(beta_lbl)
    plt.show()

    
alpha = FloatSlider(min=0.01, max=0.2, step=0.01, continuous_update=False)
mu = FloatSlider(min=-3, max=3, step=0.01, value = 1, continuous_update=False)

mu_l =  lbl(value=r'True Population \(\mu\)')
alpha_l = lbl(value=r"\(\alpha\)")
ui_power = HBox([mu_l, mu, alpha_l, alpha])

power_out = interactive_output(power_plot, {'alpha': alpha, 'mu': mu})

